In [1]:
import torch
import transformers
from ast import literal_eval
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import evaluate
from tqdm import tqdm
from peft import LoraConfig
import re

from datasets import load_dataset

pd.set_option('display.max_columns', None)

In [14]:


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
MODEL_NAME = "Qwen/Qwen3-14B"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
dataset = pd.read_csv('../../data/train.csv') 
df_with_4choices = dataset[dataset['problems'].apply(lambda x: len(literal_eval(x)['choices']) == 4)]
df_with_5choices = dataset[dataset['problems'].apply(lambda x: len(literal_eval(x)['choices']) == 5)]

In [9]:
df_with_4choices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             792 non-null    object 
 1   paragraph      792 non-null    object 
 2   problems       792 non-null    object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 30.9+ KB


In [10]:
def flatten_data(dataset: pd.DataFrame) -> pd.DataFrame:
    records = []
    for _, row in dataset.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', None),
        }
        # Include 'question_plus' if it exists
        if 'question_plus' in problems:
            record['question_plus'] = problems['question_plus']
        records.append(record)
            
    # Convert to DataFrame
    return pd.DataFrame(records)

In [11]:
df_with_4choices = flatten_data(df_with_4choices)
df_with_5choices = flatten_data(df_with_5choices)

In [ ]:
df_with_4choices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             792 non-null    object
 1   paragraph      792 non-null    object
 2   question       792 non-null    object
 3   choices        792 non-null    object
 4   answer         792 non-null    int64 
 5   question_plus  0 non-null      object
dtypes: int64(1), object(5)
memory usage: 37.2+ KB


In [22]:
# 지문 길이 기준으로 내림차순 정렬
# key=lambda x: x.str.len() -> 글자 수 기준으로 정렬하라는 뜻
sorted_df = df_with_4choices.sort_values(by='paragraph', key=lambda x: x.str.len(), ascending=False)

# 상위 5개 출력
print(sorted_df[['choices', 'paragraph', 'question']].head(5))
top_5_with_4 = sorted_df[['choices', 'paragraph', 'question']].head(5)

                                                                                    choices  \
96                                                                 [중세주의, 경험주의, 합리주의, 낭만주의]   
688     [레반트 수복을 위한 기독교 운동의 실패, 지중해 무역로의 쇠퇴, 라틴계 그리스도교인들의 콘스탄티노플 정복, 비잔틴 군에 의한 셀주크 투르크의 대패]   
153                                                     [체사레 바카리아, 장 자크 루소, 애덤 스미스, 데이비드 흄]   
647                                                       [몽골의 중동 침략, 종교개혁, 대항해 시대, 십자군 전쟁]   
631  [아바스 칼리파국 군대의 콘스탄티노플 정복, 셀주크 제곡의 아나톨리아 침공, 아바르와 불가르의 발칸 침공, 콘스탄티노플 국민의 가톨릭 그리스도교로의 개종]   

                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
top_5_with_4

,choices,paragraph,question
96,"[중세주의, 경험주의, 합리주의, 낭만주의]","캉디드는 “하지만, 존경하는 팡글로스,” “내가 어떻게 당신을 다시 볼 수 있겠습니까?”라고 말하였습니다. 팡글로스가 말하길 “당신이 나의 교수형을 본 것은 사실입니다&…외과의사가 내 시신을 사서, 집으로 가져간 뒤 해부하였습니다. 그는 배꼽부터 쇄골까지 내 몸을 십자절개하기 시작했습니다. 나보다 더 심하게 교수형을 당한 사람 없을 겁니다. 종교재판의 집행관은 차보제로 그는 사람들을화형시키는 법을 기가막힐 정도로 잘 알고 있었지만 교수형에는 익숙치 않았습니다. 줄은 젖어서 제대로 풀리지 않은 데다가 단단히 묶여 있었습니다; 한마디로, 나는 여전히 숨이 붙어 있어서, 십자절개를 할 때 끔찍한 비명을 질러 나를 해부한 외과의사를 뒤로 넘어가게 했어요&…[상세히] 내 상처를 봉합했으며; 그의 부인은 나를 간호해주었습니다. 15일이 지날 무렵 나는 회복했습니다&…. 어느날 갑자기 모스크에 들어갈 결심을 하였습니다, 그 곳에서 나이 든 이만(lman)과 주기도문을 외우는 아름다운 젊은 여신도를 보았습니다&…그녀가 부케를 떨어뜨렸으며; 나는 그것을 주어서 공손히 그녀에게 주었습니다. 그것을 전달하는 데 시간이 많이 지체되어 이만은 화가 났으며, 내가 기독교인걸 보고 그는 도움을 요청했습니다. 그들은 나를 카디 앞에 데려갔으며, 그는 내 발바닥 백 대를 때릴 것을 명령하고 나를 갤러리선으로 보냈습니다. 나는 젋은 남작과 같은 갤러리선의 같은 벤치에 묶였습니다. 이 갤러리선에는 마르세유 출신의 4명의 젋은 남성과 나폴리칸 출신의 성직자 5명, 그리고 코푸 출신의 2명의 수도승이 있었으며, 수도승들은 매일 비슷한 모험담들을 우리에게 얘기해 주었습니다. 남작은 자신이 나보다 훨씬 더 부당한 일들을 겪었다고 주장했습니다&…우리는 끊임없이 논쟁했으며, 공통적인 사건들을 연속적으로 갤러리선에 일으켰기 때문에 소의 음경으로 만든 채찍 이대를 맞았으며, 그리고 당신의 우리의 몸값을 지불할만큼 충분히 좋은 사람이었습니다.” 칸디는 “글쎄요, 존경하는 팡글로스,” “당신이 교수형에 처하고, 해부를 당하고, 채찍질을 당하고, 노를 저어야 했을 때, 당신은 항상 모든 것들이 최선을 위해 일어난다고 생각했습니까?”라고 팡글로스에게 말하였습니다. “나는 여전히 나의 처음 생각과 같습니다,”라고 팡글로스는 답하였습니다, “나는 철학자로서 내 생각을 철회할 수 없습니다, 특히 라이프니치는 결코 틀린 적이 없으며; 뿐만 아니라, 예정조화는 세상에서 가장 완벽한 것이고, 그의 플레넘(planum)과 물질 서브틸리스(materia subtilis) 또한 마찬가지입니다.” 프랑스 계몽주의 작가 봉테르의 캉디드, 1759",이 구절의 주제와 팡글로스가 그들에게 말하는 모드는 다음의 영향력을 보여줍니다
688,"[레반트 수복을 위한 기독교 운동의 실패, 지중해 무역로의 쇠퇴, 라틴계 그리스도교인들의 콘스탄티노플 정복, 비잔틴 군에 의한 셀주크 투르크의 대패]","""예루살렘과 콘스탄티노플 시 경계로부터 끔찍한 이야기가 흘러나와 우리의 귀까지 들어왔습니다. 페르시아 왕국의 한 종족, 저주 받은 종족, 신으로부터 완전히 소외된 종족이 그리스도교의 땅을 침략해 칼과 약탈, 방화로 황폐화시켰다는 것입니다. 이들은 포로 중의 일부를 자국으로 끌고갔으며 어떤 이들은 잔혹한 고문으로 죽였습니다. 신의 교회를 완전히 부숴버리거나 교회를 이들의 의식에 사용해 버렸습니다… 이제 그리스 왕국은 이제 이들에 의해 쪼개졌고 2개월의 행군으로도 건널 수 없을 만큼 방대한 영토를 빼앗겼습니다. 그렇다면 이러한 만행에 대한 복수를 하고 빼앗긴 영토를 수복하는 일을 당신이 아니면 누가 하겠습니까? 다른 나라들보다 뛰어난 당신에게 주께서는 저항하는 자들의 머릿가죽을 발 아래에 두도록 무기와, 위대한 용기, 신체 능력과 강력한 힘에 놀라운 영광을 담아 내려주셨습니다. 선조들이 행한 일들이 당신들을 감동시키고, 당신의 정신을 남자다운 업적으로 이끌게 하십시오. 이교도들의 왕국을 파괴하고 성스러운 교회의 땅을 확장한 샤를마뉴 대제와 그 아들 루이 그리고 다른 왕들의 영광과 위대함을 재연하십시오. 더러운 국가들이 차지하고 있는 우리 구세주의 거룩한 무덤과, 특히 이들의 더러움으로 무자비하게 오염된 성소들에 분노하십시오. 오, 가장 용맹한 군인들이자 무적의 선조들의 후손들이여, 타락하지 말 것이며 선조들의 용맹을 떠올리십시오. 예루살렘은 세상의 배꼽입니다. 이 땅은 마치 기쁨이 흘러넘치는 낙원처럼 다른 어떤 곳보다 비옥합니다. 인류의 구세주께서는 자신의 출현으로 영광을 드러내셨고, 거주로 아름답게 하셨으며, 고난으로 거룩하게 하셨으며, 죽음으로 구원하셨고, 장례로 영광을 나타내셨습니다. 따라서 예루살렘은 세계의 중심에 위치하고 있었지만 지금은 하느님의 원수들에게 붙잡혀 있고, 하느님을 모르는 자들과 이교도들의 숭배에 지배당하고 있습니다. 이에 예루살렘을 해방을 원하며 끝없이 도움을 바라고 있습니다. 예루살렘은 특히 당신에게 도움을 청합니다. 앞서 말한 바와 같이, 주께서는 다른 모든 나라들이 아닌 당신에게 무력으로 더 큰 영광을 내리셨기 때문입니다. 따라서 천국의 썩지 아니할 영광을 확신하면서 여러분의 죄 사함을 위한 이 여행을 떠나십시오."" 교황 우르바노 2세, 클레르몽 공의회에서 한 연설, 로버트 수도사가 기록, 1095년",다음 중 위 연설의 결과로 발생한 사건은?
153,"[체사레 바카리아, 장 자크 루소, 애덤 스미스, 데이비드 흄]","""하지만 친애하는 팡글로스."" 캉디드가 말했다. ""내가 어찌 당신을 다시 볼 수 있습니까?"" ""내가 교수형에 처해 있는 것을 당신이 본 것은 사실입니다."" 팡글로스가 말했다. ""외과 의사가 내 시신을 구입하여 집으로 가져가서 해부했지요. 배꼽부터 쇄골까지 중요한 부분을 절개하기 시작했습니다. 저보다 심한 교수형을 당한 사람은 없을 겁니다. 종교재판 집행자는 부집사로 훌륭히 화형하는 법을 알고 있었지만 교수형은 그리 익숙하지 않았습니다. 끈은 젖은데다 제대로 들어가지 않았고 게다가 묶인 상태도 좋지 않았지요. 다시 말해 중요한 부분을 절개당해 끔찍한 비명을 지르자 외과의가 등을 대고 쓰러졌습니다. 저는 아직 숨이 붙어 있었죠. [마침내 그는] 제 상처를 꿰매고 그의 아내는 절 간호해 주기까지 했습니다. 나는 15일 만에 다리를 딛고 섰습니다… 어느 날 모스크에 들어갔는데 아주 예쁘고 젊은 신도와 그녀의 후원자라는 늙은 이만이 있더군요. 그녀는 꽃다발을 떨어뜨렸습니다. 저는 꽃다발을 집어 들고 깊은 경의를 표하며 선물했습니다. 제가 선물을 전하는 데 시간이 너무 오래 걸려 이만은 화를 내기 시작했고, 제가 도움을 청하는 그리스도인임을 알았습니다. 저는 이슬람 재판관에게 끌려갔고 발바닥에 채찍질을 100대 당한 뒤 갤리선으로 보내졌습니다. 저는 젊은 남작과 똑같은 갤리선, 같은 벤치에 묶여 있었습니다. 이 갤리선에는 마르세유 출신 젊은이 4명, 나폴리 신부 5명, 코르푸 출신 수도사 2명이 타고 있었는데 매일 비슷한 모험이 일어난다고 했습니다. 남작은 저보다 더 큰 불의를 겪었다고 하더군요… 우리는 계속 논쟁했고 황소 음경으로 만든 채찍으로 스무 대를 맞았는데, 보편적인 사건이 연속되어 인격이 훌륭한 당신이 우리 갤리선에 찾아와 우리 몸값을 내주었죠."" ""음, 친애하는 팡글로스."" 캉디드가 그에게 말했다. ""그대가 교수형을 당하고, 해부당하고, 채찍에 맞고, 노를 저을 때, 항상 모든 일이 잘 풀리리라 생각했습니까?"" ""의견은 여전히 바뀌지 않았습니다."" 팡글로스가 말했다. ""저는 철학자고 철회할 수 없으니까요. 특히 라이프니츠는 결코 틀릴 수 없습니다. 게다가 사전 확립된 조화는 세상에서 가장 훌륭합니다. 그의 플레넘과 미세 물질도 그렇지요."" 프랑스 계몽주의 작가 볼테르, 캉디드, 1759년",볼테르가 캉디드를 통해 제시한 비평을 가장 긴밀히 공유하는 철학자는 누구입니까?
647,"[몽골의 중동 침략, 종교개혁, 대항해 시대, 십자군 전쟁]","""예루살렘과 콘스탄티노플 시 경계로부터 끔찍한 이야기가 흘러나와 우리의 귀까지 들어왔습니다. 페

In [24]:
# 지문 길이 기준으로 내림차순 정렬
# key=lambda x: x.str.len() -> 글자 수 기준으로 정렬하라는 뜻
sorted_df_5choices = df_with_5choices.sort_values(by='paragraph', key=lambda x: x.str.len(), ascending=False)

# 상위 5개 출력
# print(sorted_df_5choices[['choices', 'paragraph', 'question']].head(5))
top_5_with_5 = sorted_df_5choices[['choices', 'paragraph', 'question']].head(5)

In [ ]:
system_message_4 = """
당신은 객관식 문제풀이의 달인입니다.
"""

user_message_4 = """
밑의 예시와 같이 추론과정을 통해 문제를 해결하세요.
[예시1]
지문:직지 심체요절은 백운화상이 저술한 책을 청주 흥덕사에서 1377년 7월에 금속활자로 인쇄한 것이다. 1972년 '세계도서의 해'에 출품되어 세계 최고의 금속활자본으로 공인되었다. 이 책은 이러한 가치를 인정받아 2001년 9월에 유네스코 세계기록유산으로 등재되었다.
질문:다음 문화유산이 간행된 왕대에 대한 설명으로 옳은 것은?
선택지:
1. 원황실은 북쪽으로 도망가고 명이 건국되었다.
2. 기존의 토지문서를 불태워버리고 과전법을 시행하였다.
3. 원에만 권당을 설치하여 고려와 원의 지식인들이 교류하였다.
4. 명은 철령위를 설치한다고 고려에 통보하였다.
추론(CoT):
1. 지문 분석을 합니다. 간행 연도를 파악합니다. 1377년입니다.
2. 질문을 분석합니다. 간행 연도의 왕대를 분석합니다. 공민왕은 1352년부터 1374년이고, 우왕은 1374년부터 1388년입니다.
3. 선택지 대조랑 비교합니다. 3번이 정답입니다.

[예시2]
지문: 1. 다른 직장을 구하기 위해 직장을 그만둔 메리 2. 45세에 직장에서 은퇴하여 자신의 꿈을 이룬 존 3. 파트타임으로 일하고 있지만 풀타임으로 일하고 싶은 다이앤
질문:다음 중 실직자로 간주되는 사람은 누구입니까?
선택지:
1. 메리만 해당 
2. 존만 해당
3. 다이앤 해당
4. 메리와 다이앤 해당
추론(CoT):
1. 지문을 분석합니다. 메리는 직장을 직장, 즉 일자리를 그만두었으니 실업자입니다. 존은 일을 하고 있으니 실업자가 아닙니다. 다이앤은 일자리가 있으니 실업자가 아닙니다.
2. 질문의 실업자의 뜻을 정의합니다. 실업자란 일할 의사와 능력은 있으나 일자리가 없는 사람입니다.
3. 선택지를 대조합니다. 답은 1번입니다. 

[예시3]
지문: 동일한 기간에 파인애플 공급에 영향을 미치는 가뭄과 암 예방에 도움이 될 수 있는 발견이 있음
질문: 이러한 이벤트의 조합이 파인애플의 균형 가격과 수량에 어떤 영향을 주겠습니까?
선택지:
1. 증가 감소
2. 증가 불확정
3. 불확정 증가
4. 감소 불확정
추론(CoT):
1. 지문을 분석합니다 사건별 영향을 분석합니다. 가뭄이 오면 공급 감소합니다. 이는 가격 증가, 수량 감소로 이어집니다. 파인애플이 암 예방에 도움이 된다는 점이 발견됩니다. 이는 수요 증가로 이어지고 가격 증가, 수량 증가로 이어집니다.
2. 결합 효과가 발생합니다. 균형 가격은 상승 압력이고 가격 증가로 이어집니다. 균형 수량은 상쇄되고 수량 불확정으로 이어집니다.
3. 선택지를 대조합니다. 답은 1번입니다. 

### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:
"""


messages = [
            {"role": "system", "content": system_message_4},
            {"role": "user", "content": user_message_4}
        ]


SyntaxError: invalid syntax (170467232.py, line 4)

In [ ]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
)

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

NameError: name 'model' is not defined

In [ ]:
model.eval()
device = "cuda"
inputs.to(device)
output = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=False,
)

In [ ]:
len_input_prompt = len(inputs['input_ids'][0])
response = tokenizer.decode(output[0][len_input_prompt:], skip_special_tokens=True)
print(response)

In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))